<small>
Copyright (C) 2025, École Polytechnique Fédérale de Lausanne. All Rights Reserved.
</small>

---

# Exercise 5 | Offset-free Tracking

Consider the discrete-time system

$$
\begin{aligned}
x_{k+1} &= \begin{bmatrix} 0.7115 & -0.4345 \\ 0.4345 & 0.8853 \end{bmatrix} x_k + \begin{bmatrix} 0.2173 \\ 0.0573 \end{bmatrix} u_k \\
y_k &= \begin{bmatrix} 0 & 1 \end{bmatrix} x_k + d
\end{aligned}
$$

where $d$ is an unknown constant disturbance and $x_0$ is unknown. 

The goal of this exercise is to design a controller able to track a constant output reference while fulfilling input constraints

$$
-3 \leq u_k \leq 3
$$


In [ ]:
import numpy as np

A = np.array([[0.7115, -0.4345], [0.4345, 0.8853]])
B = np.array([[0.2173], [0.0573]])
C = np.array([[0, 1]])

u_max = 3.
u_min = -3.

# Dimensions
nx = B.shape[0]  # num. states
nu = B.shape[1]  # num. inputs
ny = C.shape[0]  # num. outputs
nd = 1           # num. disturbance



### Problem 1 | Observer Design

Since state and disturbances are unknown at time zero, we need to design an observer to estimate them. We call $\hat{x}$ and $\hat{d}$ the estimate of $x$ and $d$, respectively. Design an observer for the given system, and test it for the condition 
$x_0 = \begin{bmatrix} 1 & 2 \end{bmatrix}^T$, $\hat{x} = \begin{bmatrix} 3 & 0 \end{bmatrix}^T$, $\hat{d} = 0$, $d = 0.2$ with input $u = 0$.

<!-- $$x_0 = \begin{bmatrix} 1 & 2 \end{bmatrix}^\top, \hat{x} = \begin{bmatrix} 3 & 0 \end{bmatrix}^\top, \hat{d} = 0, d = 0.2, u = 0.$$ -->

**Hints:**

- You can use CVXPY to implement the MPC controller (recall exercise 4).
- To estimate the disturbance you will have to augment the state as seen in class.
- Note the eigenvalues of $(A + LC)$ are the same as those of $(A^\top + C^\top L^\top)$.
- Pole placement can be computed with the python function [`scipy.signal.place_poles`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.place_poles.html). Read the documentation for its usage.



In [ ]:
d = 0.2

# TODO: ------------------
# TODO: add your code here
# Augmented dynamics:
# x^+ = A x + B u + Bd d
# d^+ = d
# y   = C x + Cd d 
Bd = ...
Cd = ...


# Error dynamics:
# e_k = (A_hat + L @ C_hat) e_k, where e_k = [x_k - xhat_k; d_k - dhat_k]
A_hat = ...
B_hat = ...
C_hat = ...
poles = ...
# TODO: ------------------
from scipy.signal import place_poles
res = place_poles(A_hat.T, C_hat.T, poles)
L = -res.gain_matrix.T



# Test with given initial condition
num_steps = 50
x_hat_traj = np.ndarray((num_steps+1, nx))  # estimated state trajectory
d_hat_traj = np.ndarray((num_steps+1, nd))  # estimated disturbance trajectory
x_traj = np.ndarray((num_steps+1, nx))      # true state trajectory
u_traj = np.zeros((num_steps, nu))          # input trajectory

x_hat_traj[0] = np.array([3, 0])
d_hat_traj[0] = np.array([0])
x_traj[0] = np.array([1, 2])

# Simulation
for k in range(num_steps):
	# True system
	x_traj[k+1] = A @ x_traj[k]
	

	# Estimation
	# TODO: ------------------
	# TODO: add your code here
	yk = ...  # output
	...
	x_hat_traj[k+1] = ...
	d_hat_traj[k+1] = ...
	# TODO: ------------------



In [ ]:
## Visualization

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from typing import Optional
def plot_trajs(
		x_hat_traj: np.ndarray,    # estimated states trajectory
		d_hat_traj: np.ndarray,    # estimated disturbance trajectory
		x_traj: np.ndarray,        # true states trajectory
		u_traj: np.ndarray,        # input trajectory
		d: float,                  # true disturbance
		r: Optional[float] = None  # reference
		):

	fig, axs = plt.subplots(2, 2, figsize=(10, 8))

	# --- (1,1) x and x_hat ---
	axs[0, 0].plot(x_traj[:, 0], 'r', label=r'$x_1$')
	axs[0, 0].plot(x_traj[:, 1], 'b', label=r'$x_2$')
	axs[0, 0].plot(x_hat_traj[:, 0], 'r--', label=r'$\hat{x}_1$')
	axs[0, 0].plot(x_hat_traj[:, 1], 'b--', label=r'$\hat{x}_2$')
	axs[0, 0].set_title("States")
	axs[0, 0].legend()
	axs[0, 0].grid()

	# --- (1,2) y and r ---
	y_traj = x_traj @ C.T + d * np.ones_like(x_traj @ C.T)
	axs[0, 1].plot(y_traj, 'g', label=r'$y$')
	if r is not None:
		axs[0, 1].plot(r * np.ones_like(y_traj), 'k--', label=r'$r$')
	axs[0, 1].set_title("Output")
	axs[0, 1].legend()
	axs[0, 1].grid()

	# --- (2,1) u ---
	axs[1, 0].plot(u_traj, label=r'$u$')
	axs[1, 0].plot(u_max * np.ones_like(u_traj), 'g--', label=r'$u_{max}$')
	axs[1, 0].plot(u_min * np.ones_like(u_traj), 'g--', label=r'$u_{min}$')
	axs[1, 0].set_title("Input")
	axs[1, 0].legend()
	axs[1, 0].grid()

	# --- (2,2) d and d_hat ---
	axs[1, 1].plot(d * np.ones_like(d_hat_traj), '--', label=r'$d$')
	axs[1, 1].plot(d_hat_traj, label=r'$\hat{d}$')
	axs[1, 1].set_title("Disturbance")
	axs[1, 1].legend()
	axs[1, 1].grid()

	plt.tight_layout()
	plt.show()


plot_trajs(x_hat_traj, d_hat_traj, x_traj, u_traj, d)

### Problem 2 | Steady-state target computation

Given the system above, and a reference $r$, use CVXPY to compute a steady state for the system that minimizes $u^2$.

In [ ]:
import cvxpy as cp
from typing import Tuple

def compute_steady_state(d_hat: np.ndarray, r: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
	"""
	Compute the steady-state state xs and input us that minimize us^2,
	subject to the system steady-state equations and input constraints.
	"""
	d_hat = np.array(d_hat).reshape((-1,))
	r = np.array(r).reshape((-1,))

	xs_var = cp.Variable(nx, name='xs')
	us_var = cp.Variable(nu, name='us')

	# TODO: ------------------
	# TODO: add your code here
	# Objective:
	ss_obj = ...
	
	# Constraints:
	ss_cons = ...

	# TODO: ------------------

	prob = cp.Problem(cp.Minimize(ss_obj), ss_cons)
	prob.solve()
	assert prob.status == cp.OPTIMAL

	return xs_var.value, us_var.value
    
xs, us = compute_steady_state(0, 1)
print(f"The steady-state target are:\nxs = {xs}\nus = {us}")

## Problem 3 | MPC tracking
Implement an MPC controller to track an output reference signal $r$.

Confirm that the estimates converge to the true values, the output converges to the reference
and that the input does not violate the constraints by plotting the result for references $r = 1$
and $r = 0.5$. Use the same initial conditions as in problem 1.

**Hints:**

- Use a terminal set of $\mathcal{X}_f = \mathbb{R}^n$ and a terminal cost of $V_f (x_N) = \Delta x_N^\top P \Delta x_N$ where $P$ is the solution of $P−A^\top PA=Q$. 

- These correspond to a terminal controller $\pi_f(\cdot) = 0$. Note that this is a valid terminal controller
because the system is stable.

- You can use the function [`scipy.linalg.solve_discrete_lyapunov`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_discrete_lyapunov.html) to compute $P$.

- Good values for the horizon and stage costs are: $N = 5, Q= I, R = I$.

- In the previous exercise you designed an observer specifying the eigenvalues of the estimation
error state-update matrix. Eigenvalues with a small norm will speed up the estimation process,
but may increase the initial overshoot of the estimate $d$. A large $d$ can cause the problem of
computing the set-point to be infeasible. Use moderate eigenvalues (e.g. $[0.5, 0.6, 0.7]$).

In [ ]:
## MPC settings
N = 5           # horizon length
num_steps = 50  # number of steps of closed-loop simulation
Q = np.eye(2)
R = np.eye(1)


## Compute the P in terminal cost: x'Px/2
from scipy.linalg import solve_discrete_lyapunov
# TODO: ------------------
# TODO: add your code here
P = ...
# TODO: ------------------


## Formulate MPC problem (_var stands for variables, _par stands for parameters)

x_var = cp.Variable((N+1, nx), name='x')
u_var = cp.Variable((N, nu), name='u')
xs_par = cp.Parameter((nx,), name='xs')          # steady-state xs
us_par = cp.Parameter((nu,), name='us')          # steady-state us
x0_hat_par = cp.Parameter(nx, name='x0_hat')     # (estimated) initial state x0
d_hat_par = cp.Parameter(nd, 'd_hat')            # (estimated) disturbance
ref_par = cp.Parameter(ny, 'ref')                # reference


# TODO: ------------------
# TODO: add your code here
# Cost function
mpc_obj  = 0
mpc_obj += ...

# Constraints
mpc_cons = []
mpc_cons.append(...)

# TODO: ------------------

mpc_prob = cp.Problem(cp.Minimize(mpc_obj), mpc_cons)

d = np.array([0.2]) # disturbance
r = np.array([1])   # reference  # Can change it to 0.5

x_traj = np.zeros((num_steps+1, nx))
u_traj = np.zeros((num_steps, nu))
x_hat_traj = np.zeros((num_steps+1, nx))
d_hat_traj = np.zeros((num_steps+1, nd))


In [ ]:
# Initial conditions
x0 = np.array([1, 2])
x0_hat = np.array([3, 0])
d0_hat = np.array([0])

x_traj[0] = x0
x_hat_traj[0] = x0_hat
d_hat_traj[0] = d0_hat



## Closed loop simulation
for k in range(num_steps):
	xs, us = compute_steady_state(d_hat_traj[k], r)

	# TODO: ------------------
	# TODO: add your code here
	# update parameter for MPC problem
	xs_par.value = ...
	us_par.value = ...
	x0_hat_par.value = ...
	d_hat_par.value = ...
	# TODO: ------------------


	mpc_prob.solve()
	assert mpc_prob.status == cp.OPTIMAL

	# extract inputs
	u_traj[k] = u_var[0].value

	# Apply to the plant, simulate the system evolvement
	x_traj[k+1] = A @ x_traj[k] + B @ u_traj[k] + Bd @ d
    
    # TODO: ------------------
	# TODO: add your code here
    # Observer
	yk = ...   # output
	...
	x_hat_traj[k+1] = ...
	d_hat_traj[k+1] = ...
	# TODO: ------------------



In [ ]:
## Visualization
plot_trajs(x_hat_traj, d_hat_traj, x_traj, u_traj, d)